# Análisis de sentimiento con reseñas de hoteles: Parte 2
Ahora que has explorado a detalle el conjunto de datos, es momento de filtrar las columnas y luego usar técnicas de procesamiento del lenguaje natural sobre el conjunto de datos para obtener nuevos conocimientos acerca de los hoteles.

### Filtrado y operaciones de análisis de sentimiento
Como quizá ya has notado, el conjunto de datos presenta unos cuantos problemas. Algunas columnas contienen información inútil, otras parecen incorrectas. Si estas son correctas, no está claro cómo fueron calculadas, y las respuestas no pueden verificarse de forma independiente al realizar nuestros propios cálculos.

In [20]:
# Load the hotel reviews from CSV
import pandas as pd
import time
import numpy as np
# importing time so the start and end time can be used to calculate file loading time
print("Loading data file now, this could take a while depending on file size")
start = time.time()
# df is 'DataFrame' - make sure you downloaded the file to the data folder
df = pd.read_csv('C:/Users/HP/Documents/LAUNCH X/Web-dev/Projects/Machine-Learning/6-NLP/4-Reseñas_de_Hoteles_1/Data/Hotel_Reviews.csv')
df.info()
df

Loading data file now, this could take a while depending on file size
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,7.0,"[' Leisure trip ', ' Family with older childre...",704 day,48.203745,16.335677
515734,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/22/2015,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,Breakfast was ok and we got earlier check in,11,12,5.8,"[' Leisure trip ', ' Family with young childre...",712 day,48.203745,16.335677
515735,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/19/2015,8.1,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienn...,19,2823,No Positive,0,3,2.5,"[' Leisure trip ', ' Family with older childre...",715 day,48.203745,16.335677
515736,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/17/2015,8.1,Atlantis Hotel Vienna,Mexico,No Negative,0,2823,The rooms are enormous and really comfortable...,25,3,8.8,"[' Leisure trip ', ' Group ', ' Standard Tripl...",717 day,48.203745,16.335677


## Ejercicio: un poco más de procesamiento de datos
Limpia un poco más los datos. Agrega las columnas que usaremos más tarde, cambia los valores en otras columnas y elimina completamente ciertas columnas.

Procesamiento inicial de columnas

Elimina lat y lng

Reemplaza los valores de Hotel_Address con los siguientes valores (si la dirección contiene lo mismo que la ciudad y el país, cámbialo sólo a la ciudad y el país).

Estas son las únicas ciudades y países en el conjunto de datos:

Amsterdam, Netherlands

Barcelona, Spain

London, United Kingdom

Milan, Italy

Paris, France

Vienna, Austria

In [21]:
def replace_address(row):
    if "Netherlands" in row["Hotel_Address"]:
        return "Amsterdam, Netherlands"
    elif "Barcelona" in row["Hotel_Address"]:
        return "Barcelona, Spain"
    elif "United Kingdom" in row["Hotel_Address"]:
        return "London, United Kingdom"
    elif "Milan" in row["Hotel_Address"]:        
        return "Milan, Italy"
    elif "France" in row["Hotel_Address"]:
        return "Paris, France"
    elif "Vienna" in row["Hotel_Address"]:
        return "Vienna, Austria" 

# Replace all the addresses with a shortened, more useful form
df["Hotel_Address"] = df.apply(replace_address, axis = 1)
# The sum of the value_counts() should add up to the total number of reviews
print(df["Hotel_Address"].value_counts())

Hotel_Address
London, United Kingdom    262301
Barcelona, Spain           60149
Paris, France              59928
Amsterdam, Netherlands     57214
Vienna, Austria            38939
Milan, Italy               37207
Name: count, dtype: int64


Ahora puedes consultar datos a nivel de país:

In [22]:
display(df.groupby("Hotel_Address").agg({"Hotel_Name": "nunique"}))

,Hotel_Name
Hotel_Address,
"Amsterdam, Netherlands",105
"Barcelona, Spain",211
"London, United Kingdom",400
"Milan, Italy",162
"Paris, France",458
"Vienna, Austria",158


Procesa las columnas Hotel Meta-review

Elimina Additional_Number_of_Scoring

Reemplaza Total_Number_of_Reviews con el número total de reseñas para ese hotel que realmente están en el conjunto de datos

Reemplaza Average_Score con nuestros propio puntaje calculado

In [23]:
# Calcula el total de revisiones por hotel
total_reviews_per_hotel = df.groupby('Hotel_Name')['Hotel_Name'].transform('count')

# Asigna los valores calculados a una nueva columna llamada Total_Number_of_Reviews
df['Total_Number_of_Reviews'] = total_reviews_per_hotel

# Calcula el promedio de puntuación por hotel y redondea a una decimal
average_score_per_hotel = df.groupby('Hotel_Name')['Reviewer_Score'].transform('mean').round(1)

# Asigna los valores calculados a una nueva columna llamada Average_Score
df['Average_Score'] = average_score_per_hotel

In [24]:
# Verifica si la columna 'Additional_Number_of_Scoring' existe antes de intentar eliminarla
if 'Additional_Number_of_Scoring' in df.columns:
    df.drop(["Additional_Number_of_Scoring"], axis=1, inplace=True)
else:
    print("La columna 'Additional_Number_of_Scoring' no se encontró en el DataFrame.")

# Calcula el total de revisiones por hotel
total_reviews_per_hotel = df.groupby('Hotel_Name')['Hotel_Name'].transform('count')

# Asigna los valores calculados a una nueva columna llamada Total_Number_of_Reviews
df['Total_Number_of_Reviews'] = total_reviews_per_hotel

# Calcula el promedio de puntuación por hotel y redondea a una decimal
average_score_per_hotel = df.groupby('Hotel_Name')['Reviewer_Score'].transform('mean').round(1)

# Asigna los valores calculados a una nueva columna llamada Average_Score
df['Average_Score'] = average_score_per_hotel

Al parecer la columna `Additional_Number_of_Scoring` ya ha sido eliminada. 
El mensaje indica que no se encontró la columna que intentaste eliminar, lo cual es normal si ya ha sido eliminada o si el nombre de la columna no coincide exactamente con lo que estás proporcionando en la lista de columnas a eliminar.

In [25]:
if 'Additional_Number_of_Scoring' in df.columns:
    df.drop(["Additional_Number_of_Scoring"], axis=1, inplace=True)
    print("La columna 'Additional_Number_of_Scoring' se eliminó del DataFrame.")
else:
    print("La columna 'Additional_Number_of_Scoring' no se encontró en el DataFrame.")

La columna 'Additional_Number_of_Scoring' no se encontró en el DataFrame.


Verificamos que la columna `Additional_Number_of_Scoring` ha sido eliminada.

#### Filtrar las etiquetas
Recuerda que el objetivo de los conjuntos de datos es agregar sentimiento y columnas que nos ayudarán a elegir el mejor hotel (para ti o quizá un cliente te pidió realizar un bot de recomendación de hoteles). Necesitas preguntarte si la etiquetas son útiles o no en el conjunto de datos final. Aquí hay una interpretación (si necesitabas el conjunto de datos por otras razones, distintas etiquetas podrían permanecer dentro/fuera de la selección):

    1. El tipo de viaje es relevante, y así debería seguir
    2. El tipo de grupo de huéspedes es importante, y así debería seguir
    3. El tipo de cuarto, suite o estudio en que se quedó el huésped es irrelevante (todos los hoteles tienen básicamente los mismos cuartos)
    4. El dispositivo desde el cual se envió la reseña es irrelevante
    5. El número de noches que se hospedó el crítico podría ser relevante si atribuiste a las estancias largas con mayor gusto por el hotel, pero es exagerado, y probablemente irrelevante
En resumen, **conserva 2 tipos de etiquetas y elimina el resto**.

Primero, no quieres contar las etiquetas hasta que estén en un mejor formato, lo cual significa eliminar los corchetes y comillas. Puedes hacerlo de varias formas, pero quieres la más rápida ya que podría tomar un largo tiempo el procesar demasiados datos. Afortunadamente, pandas tiene una forma fácil de hacer cada uno de estos pasos.

In [26]:
# Remove opening and closing brackets
df.Tags = df.Tags.str.strip("[']")
# remove all quotes too
df.Tags = df.Tags.str.replace(" ', '", ",", regex = False)

df.Tags

0          Leisure trip, Couple, Duplex Double Room, Sta...
1          Leisure trip, Couple, Duplex Double Room, Sta...
2          Leisure trip, Family with young children, Dup...
3          Leisure trip, Solo traveler, Duplex Double Ro...
4          Leisure trip, Couple, Suite, Stayed 2 nights,...
                                ...                        
515733     Leisure trip, Family with older children, 2 r...
515734     Leisure trip, Family with young children, Sta...
515735     Leisure trip, Family with older children, 2 r...
515736     Leisure trip, Group, Standard Triple Room, St...
515737     Leisure trip, Family with young children, 2 r...
Name: Tags, Length: 515738, dtype: object

Algunas de las etiquetas comunes como `Submitted from a mobile device` no nos son útiles, por lo que podría ser una buena idea quitarlas antes de contar la ocurrencia de las frases, pero es una operación tan rápida que puedes dejarlos e ignorarlos.

#### Eliminando la longitud de las etiquetas de estadía
Eliminar estas etiquetas es el paso 1, reduce levemente el número total de etiquetas a ser consideradas. Nota que no las eliminas del conjunto de datos, sólo eliges eliminarlas de la consideración de valores a contar/mantener en el conjunto de datos de reseñas.

<table>
  <tr>
    <th>Length of stay</th>
    <th>Count</th>
  </tr>
  <tr>
    <td>Stayed 1 night</td>
    <td>193645</td>
  </tr>
  <tr>
    <td>Stayed 2 nights</td>
    <td>133937</td>
  </tr>
  <tr>
    <td>Stayed 3 nights</td>
    <td>95821</td>
  </tr>
  <tr>
    <td>Stayed 4 nights</td>
    <td>47817</td>
  </tr>
  <tr>
    <td>Stayed 5 nights</td>
    <td>20845</td>
  </tr>
  <tr>
    <td>Stayed 6 nights</td>
    <td>9776</td>
  </tr>
  <tr>
    <td>Stayed 7 nights</td>
    <td>7399</td>
  </tr>
  <tr>
    <td>Stayed 8 nights</td>
    <td>2502</td>
  </tr>
  <tr>
    <td>Stayed 9 nights</td>
    <td>1293</td>
  </tr>
  <tr>
    <td>...</td>
    <td>...</td>
  </tr>
  <!-- Agrega más filas según sea necesario -->
</table>

Existe una gran variedad de cuartos, suites, studios, departamentos, y así sucesivamente. Estos significan casi lo mismo y no te son relevantes, así que elimínalos de consideración.

<table>
  <tr>
    <th>Type of room</th>
    <th>Count</th>
  </tr>
  <tr>
    <td>Double Room</td>
    <td>35207</td>
  </tr>
  <tr>
    <td>Standard Double Room</td>
    <td>32248</td>
  </tr>
  <tr>
    <td>Superior Double Room</td>
    <td>31393</td>
  </tr>
  <tr>
    <td>Deluxe Double Room</td>
    <td>24823</td>
  </tr>
  <tr>
    <td>Double or Twin Room</td>
    <td>22393</td>
  </tr>
  <tr>
    <td>Standard Double or Twin Room</td>
    <td>17483</td>
  </tr>
  <tr>
    <td>Classic Double Room</td>
    <td>16989</td>
  </tr>
  <tr>
    <td>Superior Double or Twin Room</td>
    <td>13570</td>
  </tr>
</table>

Finalmente, y esto es placentero (porque no requirió casi nada de procesamiento), te quedarás con las siguientes etiquetas útiles:

<table>
  <tr>
    <th>Tag</th>
    <th>Count</th>
  </tr>
  <tr>
    <td>Leisure trip</td>
    <td>417778</td>
  </tr>
  <tr>
    <td>Couple</td>
    <td>252294</td>
  </tr>
  <tr>
    <td>Solo traveler</td>
    <td>108545</td>
  </tr>
  <tr>
    <td>Business trip</td>
    <td>82939</td>
  </tr>
  <tr>
    <td>Group (combined with Travellers with friends)</td>
    <td>67535</td>
  </tr>
  <tr>
    <td>Family with young children</td>
    <td>61015</td>
  </tr>
  <tr>
    <td>Family with older children</td>
    <td>26349</td>
  </tr>
  <tr>
    <td>With a pet</td>
    <td>1405</td>
  </tr>
</table>

Podrías argumentar que Travellers with friends es casi lo mismo que Group, y sería justo combinar las dos de arriba. El código para identificar las etiquetas correctas es el notebook de Etiquetas.

El último paso es crear nuevas columnas para cada una de estas etiquetas. Luego, para cada fila de reseña, si la columna Tag coincide con una de las nuevas columnas, agrega un 1, si no es así, agrega un 0. El resultado final será un conteo de cuántos críticos eligieron este hotel (en agregado), digamos, para negocios vs ocio, o para traer mascota, y esta es información útil al recomendar un hotel.

In [27]:
# Process the Tags into new columns
# The file Hotel_Reviews_Tags.py, identifies the most important tags
# Leisure trip, Couple, Solo traveler, Business trip, Group combined with Travelers with friends, 
# Family with young children, Family with older children, With a pet
df["Leisure_trip"] = df.Tags.apply(lambda tag: 1 if "Leisure trip" in tag else 0)
df["Couple"] = df.Tags.apply(lambda tag: 1 if "Couple" in tag else 0)
df["Solo_traveler"] = df.Tags.apply(lambda tag: 1 if "Solo traveler" in tag else 0)
df["Business_trip"] = df.Tags.apply(lambda tag: 1 if "Business trip" in tag else 0)
df["Group"] = df.Tags.apply(lambda tag: 1 if "Group" in tag or "Travelers with friends" in tag else 0)
df["Family_with_young_children"] = df.Tags.apply(lambda tag: 1 if "Family with young children" in tag else 0)
df["Family_with_older_children"] = df.Tags.apply(lambda tag: 1 if "Family with older children" in tag else 0)
df["With_a_pet"] = df.Tags.apply(lambda tag: 1 if "With a pet" in tag else 0)


#### Guarda tu archivo
Finalmente, guarda el conjunto de datos tal cual se encuentra ahora con un nuevo nombre.

In [28]:
df.drop(["Review_Total_Negative_Word_Counts", "Review_Total_Positive_Word_Counts", "days_since_review", "Total_Number_of_Reviews_Reviewer_Has_Given"], axis = 1, inplace=True)

# Saving new data file with calculated columns
print("Saving results to Hotel_Reviews_Filtered.csv")
df.to_csv(r'./Data/Hotel_Reviews_Filtered.csv', index = False)

Saving results to Hotel_Reviews_Filtered.csv


### Operaciones de análisis de sentimiento

En esta sección final, aplicarás análisis de sentimiento a las columnas de reseña y guardarás los resultados en un conjunto de datos.

### Ejercicio: Carga y guarda los datos filtrados
Nota que ahora estás cargando el conjunto de datos filtrado que fue guardado en la sección anterior, no el conjunto de datos original.

In [31]:
pip install nltk

In [34]:
import time
import pandas as pd
import nltk as nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')

# Load the filtered hotel reviews from CSV
df = pd.read_csv('./Data/Hotel_Reviews_Filtered.csv')
df

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


,Hotel_Address,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Total_Number_of_Reviews,Positive_Review,Reviewer_Score,Tags,lat,lng,Leisure_trip,Couple,Solo_traveler,Business_trip,Group,Family_with_young_children,Family_with_older_children,With_a_pet
0,"Amsterdam, Netherlands",8/3/2017,7.8,Hotel Arena,Russia,I am so angry that i made this post available...,405,Only the park outside of the hotel was beauti...,2.9,"Leisure trip, Couple, Duplex Double Room, Sta...",52.360576,4.915968,1,1,0,0,0,0,0,0
1,"Amsterdam, Netherlands",8/3/2017,7.8,Hotel Arena,Ireland,No Negative,405,No real complaints the hotel was great great ...,7.5,"Leisure trip, Couple, Duplex Double Room, Sta...",52.360576,4.915968,1,1,0,0,0,0,0,0
2,"Amsterdam, Netherlands",7/31/2017,7.8,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,405,Location was good and staff were ok It is cut...,7.1,"Leisure trip, Family with young children, Dup...",52.360576,4.915968,1,0,0,0,0,1,0,0
3,"Amsterdam, Netherlands",7/31/2017,7.8,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,405,Great location in nice surroundings the bar a...,3.8,"Leisure trip, Solo traveler, Duplex Double Ro...",52.360576,4.915968,1,0,1,0,0,0,0,0
4,"Amsterdam, Netherlands",7/24/2017,7.8,Hotel Arena,New Zealand,You When I booked with your company on line y...,405,Amazing location and building Romantic setting,6.7,"Leisure trip, Couple, Suite, Stayed 2 nights,...",52.360576,4.915968,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,"Vienna, Austria",8/30/2015,7.8,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,325,location,7.0,"Leisure trip, Family with older children, 2 r...",48.203745,16.335677,1,0,0,0,0,0,1,0
515734,"Vienna, Austria",8/22/2015,7.8,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,325,Breakfast was ok and we got earlier check in,5.8,"Leisure trip, Family with young children, Sta...",48.203745,16.335677,1,0,0,0,0,1,0,0
515735,"Vienna, Austria",8/19/2015,7.8,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienn...,325,No Positive,2.5,"Leisure trip, Family with older children, 2 r...",48.203745,16.335677,1,0,0,0,0,0,1,0
515736,"Vienna, Austria",8/17/2015,7.8,Atlantis Hotel Vienna,Mexico,No Negative,325,The rooms are enormous and really comfortable...,8.8,"Leisure trip, Group, Standard Triple Room, St...",48.203745,16.335677,1,0,0,0,1,0,0,0


#### Eliminando stop words
Si fueses a ejecutar el análisis de sentimiento en las columnas de reseñas positivas y negativas, te llevaría mucho tiempo. Al probarlo en un laptop de pruebas poderosa con una CPU rápida, tomó de 12 - 14 minutos dependiendo en qué biblioteca de sentimientos fue usada. Eso es demasiado tiempo (relativamente), por lo que vale la pena investigar si se puede acelerar.

Eliminar stop words, o palabras comunes del Inglés que no cambian el sentimiento de una oración, es el primer paso. Al eliminarlas, el análisis de sentimiento debería ejecutarse más rápido, pero no sería menos preciso (ya que las stop words no afectan el sentimiento, pero sí ralentizan el análisis).

La reseña negativa más larga fue de 395 palabras, pero después de eliminar las stop words, es de 195 palabras.

Eliminar las stops words también es una operación rápida, remover las stop words de 2 columnas de reseñas de 515,000 filas tomó 3.3 segundos en el dispositivo de prueba, Podría tomar ligeramente más o menos tiempo dependiendo de la velocidad del CPU de tu dispositivo, la RAM, si tienes un disco de estado sólido o no, entre otros factores. La relatividad de corto período de operación significa que si se mejora el tiempo de análisis de sentimiento, entonces vale la pena hacerlo.

In [35]:
# Remove stop words - can be slow for a lot of text!
# Ryan Han (ryanxjhan on Kaggle) has a great post measuring performance of different stop words removal approaches
# https://www.kaggle.com/ryanxjhan/fast-stop-words-removal # using the approach that Ryan recommends
start = time.time()
cache = set(stopwords.words("english"))
def remove_stopwords(review):
    text = " ".join([word for word in review.split() if word not in cache])
    return text

# Remove the stop words from both columns
df.Negative_Review = df.Negative_Review.apply(remove_stopwords)   
df.Positive_Review = df.Positive_Review.apply(remove_stopwords)

### Realizando análisis de sentimiento
Ahora deberías calcular el análisis de sentimiento tanto para las columnas de reseñas positivas como negativas y almacenar el resultado en 2 nuevas columnas. La prueba del sentimiento será compararlo con el puntaje del crítico para la misma reseña. Por ejemplo, si el sentimiento piensa que la reseña negativa tuvo un sentimiento de 1 (sentimiento extremadamente positivo) y un sentimiento de reseña positiva de 1, pero el crítico le dió al hotel el puntaje más bajo posible, luego que el texto de la reseña no coincide con el puntaje o el analizador de sentimiento no pudo reconocer el sentimiento de forma correcta. Deberías esperar que algunos puntajes de sentimiento sean completamente erróneos, y a menudo será explicable, por ejemplo, la reseña podría ser extremadamente sarcástica "Of course I LOVED sleeping in a room with no heating" y el analizador de sentimiento piensa que es un sentimiento positivo, aunque un humano leyéndolo sabría que fue sarcasmo.

NLTK provee distintos analizadores de sentimiento con los cuales aprender, y puedes sustituirlos y así ver si el sentimiento es más o menos preciso. El análisis de sentimiento VADER se usó aquí.

> Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

In [36]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create the vader sentiment analyser (there are others in NLTK you can try too)
vader_sentiment = SentimentIntensityAnalyzer()
# Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

# There are 3 possibilities of input for a review:
# It could be "No Negative", in which case, return 0
# It could be "No Positive", in which case, return 0
# It could be a review, in which case calculate the sentiment
def calc_sentiment(review):    
    if review == "No Negative" or review == "No Positive":
        return 0
    return vader_sentiment.polarity_scores(review)["compound"]    

Después en el programa cuando este listo para calcular el sentimiento, se puede aplicar a cada reseña como sigue:

In [37]:
# Add a negative sentiment and positive sentiment column
print("Calculating sentiment columns for both positive and negative reviews")
start = time.time()
df["Negative_Sentiment"] = df.Negative_Review.apply(calc_sentiment)
df["Positive_Sentiment"] = df.Positive_Review.apply(calc_sentiment)
end = time.time()
print("Calculating sentiment took " + str(round(end - start, 2)) + " seconds")

Calculating sentiment columns for both positive and negative reviews
Calculating sentiment took 269.86 seconds


Esto puede tomar un tiempo  para realizar este proceso dependindo de cada equipo. Si quieres imprimir los resultados y ver si el sentimiento coincide con la reseña:

In [38]:
df = df.sort_values(by=["Negative_Sentiment"], ascending=True)
print(df[["Negative_Review", "Negative_Sentiment"]])
df = df.sort_values(by=["Positive_Sentiment"], ascending=True)
print(df[["Positive_Review", "Positive_Sentiment"]])

                                          Negative_Review  Negative_Sentiment
186584  So bad experience memories I hotel The first n...             -0.9920
129503  First charged twice room booked booking second...             -0.9896
307286  The staff Had bad experience even booking Janu...             -0.9889
201953  Everything DO NOT STAY AT THIS HOTEL I never i...             -0.9886
452092  No WLAN room Incredibly rude restaurant staff ...             -0.9884
...                                                   ...                 ...
138365  Wifi terribly slow I speed test network upload...              0.9938
79215   I find anything hotel first I walked past hote...              0.9938
278506  The property great location There bakery next ...              0.9945
339189  Guys I like hotel I wish return next year Howe...              0.9948
480509  I travel lot far visited countless number hote...              0.9957

[515738 rows x 2 columns]
                                     

¡Lo último por hacer con el archivo antes de usarlo, es guardarlo! También se puede considerar reordenar todas las nuevas columnas para que sean fáciles de usar (para un humano, es un cambio cosmético).

In [39]:
# Reorder the columns (This is cosmetic, but to make it easier to explore the data later)
df = df.reindex(["Hotel_Name", "Hotel_Address", "Total_Number_of_Reviews", "Average_Score", "Reviewer_Score", "Negative_Sentiment", "Positive_Sentiment", "Reviewer_Nationality", "Leisure_trip", "Couple", "Solo_traveler", "Business_trip", "Group", "Family_with_young_children", "Family_with_older_children", "With_a_pet", "Negative_Review", "Positive_Review"], axis=1)

print("Saving results to Hotel_Reviews_NLP.csv")
df.to_csv(r"./Data/Hotel_Reviews_NLP.csv", index = False)

Saving results to Hotel_Reviews_NLP.csv


### Conclusión
Cuando se inicio, teniamos un conjunto de datos con columnas y datos pero no todos ellos podían ser verificados o usados. Exploramos los datos, se filtro lo que no se necesita, se convirtieron etiquetas en algo útil, calculamos los propios promedios, agregamos algunas columnas de sentimiento, todo con tal de aprender cosas interesantes acerca de procesar texto natural.